In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd

from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

one_hot_enc = OneHotEncoder()

/home/umcr7/ml/env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
X = pd.read_csv('train.csv') 
Y = X['label'].reshape(-1, 1)
X.drop(columns='label', inplace=True)
X = X/255
print(X.shape)
print(Y.shape)

/home/umcr7/ml/env/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


(42000, 784)
(42000, 1)


In [4]:
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.10, random_state=42)
y_train = one_hot_enc.fit_transform(y_train.reshape(-1, 1)).toarray()
y_val = one_hot_enc.fit_transform(y_val.reshape(-1, 1)).toarray()
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)


(37800, 784) (37800, 10)
(4200, 784) (4200, 10)


In [5]:
test = pd.read_csv('test.csv') / 255
print(test.shape)

(28000, 784)


In [6]:
weights = {
#     'wc1': tf.get_variable('W0', shape=(3,3,1,32), initializer=tf.contrib.layers.xavier_initializer()), 
#     'wc2': tf.get_variable('W1', shape=(3,3,32,64), initializer=tf.contrib.layers.xavier_initializer()), 
#     'wc3': tf.get_variable('W2', shape=(3,3,64,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'wd1': tf.get_variable('W3', shape=(4*4*128,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'out': tf.get_variable('W6', shape=(128,10), initializer=tf.contrib.layers.xavier_initializer()), 
}
biases = {
#     'bc1': tf.get_variable('B0', shape=(32), initializer=tf.contrib.layers.xavier_initializer()),
#     'bc2': tf.get_variable('B1', shape=(64), initializer=tf.contrib.layers.xavier_initializer()),
#     'bc3': tf.get_variable('B2', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'bd1': tf.get_variable('B3', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'out': tf.get_variable('B4', shape=(10), initializer=tf.contrib.layers.xavier_initializer()),
}


# creating placeholders
X_train = tf.placeholder(tf.float32, shape=[None, 784])
Y_train = tf.placeholder(tf.float32, shape=[None, 10])
learning_rate = tf.placeholder(tf.float32, shape=[])

def conv_net(X_train, weights, biases):
  input_layer = tf.reshape(X_train, [-1, 28, 28, 1])  #X_train.reshape(-1, 28, 28, 1)
  conv1 = tf.layers.conv2d(input_layer, filters=32,kernel_size=[3, 3],padding="same",activation=tf.nn.relu, bias_initializer=tf.zeros_initializer())
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2, padding='SAME')
  
  conv2 = tf.layers.conv2d(pool1,  filters=64,kernel_size=[3, 3],padding="same",activation=tf.nn.relu, bias_initializer=tf.zeros_initializer())
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2, padding='SAME')
  
  conv3 = tf.layers.conv2d(pool2, filters=128,kernel_size=[3, 3],padding="same",activation=tf.nn.relu, bias_initializer=tf.zeros_initializer())
  pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2, padding='SAME')  
  
  fc1 = tf.reshape(pool3, [-1, 4 * 4 * 128]) #fully connected layer
  fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
  fc1 = tf.nn.relu(fc1)
  
  out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])  
  return out
  
  
pred = conv_net(X_train, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y_train))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


batch_size = 128
init = tf.global_variables_initializer()
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y_train, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))




Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [8]:
# For visulaizing in tesnorflow 
tf.summary.scalar('mse', cost)
summary_ops = tf.summary.merge_all()
train_writer = tf.summary.FileWriter('./MnistLog')

In [ ]:
################################ execution of graph #####################################


with tf.Session() as sess:
    sess.run(init) 
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    for i in range(20):
        for batch in range(len(x_train)//batch_size):
            batch_x = x_train[batch*batch_size:min((batch+1)*batch_size,len(x_train))]
            batch_y = y_train[batch*batch_size:min((batch+1)*batch_size,len(y_train))]    
            # Run optimization op (backprop).
                # Calculate batch loss and accuracy
            opt = sess.run(optimizer, feed_dict={X_train: batch_x, Y_train: batch_y, learning_rate:0.002})
            loss, acc, summary = sess.run([cost, accuracy, summary_ops], feed_dict={X_train: batch_x, Y_train: batch_y, learning_rate:0.002})
        train_writer.add_summary(summary, i) 
        print("Epoch " + str(i) + ", Loss= " + \
                      "{:.6f}".format(loss) + ", Training Accuracy= " + \
                      "{:.5f}".format(acc))
#         print("Optimization Finished!")

        # Calculate accuracy for all 10000 mnist test images
        test_acc,valid_loss = sess.run([accuracy,cost], feed_dict={X_train: x_val, Y_train: y_val})
        train_loss.append(loss)
        test_loss.append(valid_loss)
        train_accuracy.append(acc)
        test_accuracy.append(test_acc)
        print("Testing Accuracy:","{:.5f}".format(test_acc))
    
    print("predicting on test data")
    pred_list = sess.run(pred, feed_dict = {X_train: test})
        
  

Epoch 0, Loss= 0.013681, Training Accuracy= 0.99219
Testing Accuracy: 0.97976
Epoch 1, Loss= 0.003956, Training Accuracy= 1.00000
Testing Accuracy: 0.98500


In [13]:
len(pred_list)

28000

In [0]:
submission = pd.DataFrame(np.argmax(pred_list, 1), columns=['Label'])

In [0]:
submission.index+=1

In [0]:
submission.index.name = 'ImageId'

In [33]:
submission

,Label
ImageId,
1,2
2,0
3,9
4,9
5,3
6,7
7,0
8,3
9,0


In [0]:
submission.to_csv('DigitRecognizerConvnetTF.csv')

In [0]:
from google.colab import files

In [0]:
files.download('DigitRecognizerConvnetTF.csv')